# Basic Tensor Operation

In [1]:
# Totally wrong, refer to CartesianIndices
function kron(t1, t2)
    if ndims(t1) != ndims(t2)
        error("ndims t1 must be equaled to t2")
    end
    
    ret = zeros(length(t1) * length(t2))
    for i in 1:length(t1)
        ret[(i-1)*length(t2)+1:i*length(t2)] = t1[CartesianIndices(t1)[i]] * t2
    end
    dims = size(t1).*size(t2)
    return reshape(ret, (dims))
end

kron (generic function with 1 method)

In [2]:
σ₁ = [0 1;1 0]
σ₃ = [1 0;0 -1]
kron(σ₁, σ₃)

4×4 Array{Float64,2}:
 0.0   1.0   1.0  0.0
 0.0   0.0   0.0  0.0
 0.0   0.0   0.0  0.0
 0.0  -1.0  -1.0  0.0

##### PROBLEM
1. CartersianIndices生成的顺序与平常使用的遍历顺序并不一致，并不是首先遍历完第一维，再继续往下遍历

In [3]:
# The right version for kron
function ⨂(t1::AbstractArray, t2::AbstractArray)
    if ndims(t1) != ndims(t2)
        error("ndims t1 must be equaled to t2")
    end
    
    T = promote_type(eltype(t2), eltype(t1))
    ret = zeros(T, size(t1).*size(t2))
    for i in CartesianIndices(t1)
        c_start = CartesianIndex(.+(.*(.-(Tuple(i), 1), size(t2)), 1))
        c_end = CartesianIndex(Tuple(i).*size(t2))
        ret[c_start:c_end] = t1[i] * t2
    end
    return ret
end

⨂ (generic function with 1 method)

In [4]:
⨂(σ₁, σ₃)

4×4 Array{Int64,2}:
 0   0  1   0
 0   0  0  -1
 1   0  0   0
 0  -1  0   0

In [5]:
function contract!(a::AbstractArray, b::AbstractArray, dima::Tuple, dimb::Tuple)
    if [size(a)[x] for x in dima] != [size(b)[x] for x in dimb] error("size is wrong") end
    
    a_left = filter(x -> !(x in dima), [x for x in 1:ndims(a)])
    a_right = collect(dima)
    a_perm = [a_left; a_right]
    a_reshape_size = [size(a)[x] for x in a_left]
    a_left_len = prod(a_reshape_size)
    a_right_len = prod([size(a)[x] for x in a_right])
    
    b_left = collect(dimb)
    b_right = filter(x -> !(x in dimb), [x for x in 1:ndims(b)])
    b_perm = [b_left; b_right]
    b_left_len = prod([size(b)[x] for x in b_left])
    b_reshape_size = [size(b)[x] for x in b_right]
    b_right_len = prod(b_reshape_size)
    
    am = permutedims(a, a_perm)
    bm = permutedims(b, b_perm)
    
    ra = reshape(a, (a_left_len, a_right_len))
    rb = reshape(b, (b_left_len, b_right_len))
    ret = ra*rb
    
    return reshape(ret, Tuple([a_reshape_size; b_reshape_size]))
end

contract! (generic function with 1 method)

In [6]:
a = rand(3,4,3,6,6,8)
b = rand(3,4,6,3,3,8)
dima = (1,3,4)
dimb = (1,5,3)
contract!(a, b, dima, dimb)

4×6×8×4×3×8 Array{Float64,6}:
[:, :, 1, 1, 1, 1] =
 13.1518  13.2516  14.4384  14.3201  12.9842  14.5912
 14.4724  13.4575  11.5543  13.8214  14.3049  14.1966
 14.449   12.4142  15.7     15.0041  15.0333  14.4058
 12.6353  14.0363  12.4949  13.8924  14.0802  14.6432

[:, :, 2, 1, 1, 1] =
 12.8189  14.1672  13.0459  12.1002  10.7741  13.2698
 13.5071  12.3864  12.5421  13.7285  14.3351  14.6629
 14.4197  15.6196  14.1348  10.2711  13.8605  15.3771
 15.5885  15.1429  13.7351  13.0248  14.8973  14.1335

[:, :, 3, 1, 1, 1] =
 14.9784  12.4225  13.3236  14.0696  13.1426  12.7352
 12.8427  12.3523  14.4052  13.3852  12.7356  13.7081
 16.3325  13.5416  13.905   13.8778  12.0486  14.604 
 14.789   11.3531  13.0465  13.0456  12.7478  12.5622

[:, :, 4, 1, 1, 1] =
 13.8755  11.485   14.3048  12.4752  13.2314  13.0228
 14.0245  13.297   15.5274  13.5882  15.0941  14.3081
 15.0126  14.474   15.3361  13.2366  13.1916  13.2513
 13.2336  14.1919  13.4854  12.6348  14.3265  14.5388

[:, :, 5, 1, 1, 1]

##### PROBLEM
1. Array(tuple)错误，只能用[x for x in tuple]或者[x...]或者collect(tuple)
2. prod(A)求所有元素的积


In [7]:
# direct sum
function ⨁(𝑨::AbstractArray{𝕋, ℕ}, 𝑩::AbstractArray{𝕋, ℕ}, axes::Tuple) where {𝕋, ℕ}
    𝑪 = []
    
    s𝑨 = size(𝑨) # store size of 𝑨 and 𝑩 to calculate size of 𝑪
    s𝑩 = size(𝑩)
    s𝑪 = []
    
    if length(axes) == ℕ
        s𝑪 = [x+y for (x, y) in zip(s𝑨, s𝑩)]
        𝑪 = zeros(s𝑪...)
        𝑩start = CartesianIndex([x+1 for x in s𝑨]...)
        𝑩end = CartesianIndex(s𝑪...)
    else
        if sum([[x!=y for (x, y) in zip(s𝑨, s𝑩)][x] for x in axes]) != 0
            error("parameter wrong")
        end
        
        # if x in axes, this dim's data will not expand
        #for x in 1:N if x in axes push!(s𝑪, s𝑨[x]) else push!(s𝑪, s𝑨[x]+s𝑩[x]) end end
        [x in axes ? s𝑪 : s𝑨[x]+s𝑩[x] for x in 1:N] # NOTES:P2
        𝑪 = zeros(s𝑪...)
        
        # construct 𝑩start
        tmp = ones(Int64, N) # NOTES:P1
        for x in 1:N if !(x in axes) tmp[x]=1+s𝑨[x] end end
        𝑩start = CartesianIndex((tmp...,))
        𝑩end = CartesianIndex(s𝑪...)
    end
    
    𝑨start = CartesianIndices(𝑪)[1]
    𝑨end = CartesianIndex(s𝑨...)
    𝑪[𝑨start:𝑨end] = 𝑨
    𝑪[𝑩start:𝑩end] = 𝑩
    
    return 𝑪
end

⨁ (generic function with 1 method)

In [8]:
a = [1 2;3 4]
b = [2 3;5 6]
⨁(a, b, (1, 2,))

4×4 Array{Float64,2}:
 1.0  2.0  0.0  0.0
 3.0  4.0  0.0  0.0
 0.0  0.0  2.0  3.0
 0.0  0.0  5.0  6.0

##### PROBLEM
1. 在构造CartesianIndex的时候，需要注意zeros的默认类型是Float64，因此，注意在P1处需要特别添加参数Int64
2. 对于?:，两个符号的前后都必须要用空格分隔开

In [9]:
# teacher guo's version
function directsum(a::Tensor{Ta, N}, b::Tensor{Tb, N}, axs) where {Ta, Tb, N}
 isempty(a) && return b
 isempty(b) && return a
 rk = N
 dimc = Vector{Int}(undef, rk)
 dim = [1 for i=1:rk]
    shape_a = shape(a)
    shape_b = shape(b)
 for i=1:rk
        if i in axs
            dim[i] = shape_a[i]+1
            dimc[i] = shape_a[i] + shape_b[i]
        else
            dimc[i] = shape_a[i]
            (shape_a[i] == shape_b[i]) || error("shape mismatch.")
        end
 end
    c = zeros(promote_type(Ta, Tb), dimc...)
    c[[1:shape_a[i] for i=1:rk]...] = a
    c[[dim[i]:dimc[i] for i=1:rk]...] = b
    return c
end

UndefVarError: UndefVarError: Tensor not defined